In [1]:
import re
import sys
from time import sleep
from pprint import pprint
from pathlib import Path
from functools import partial
import subprocess
from subprocess import PIPE

import numpy as np
import optuna
import pandas as pd
import matplotlib.pyplot as plt

FILENAME = "main.cpp"

In [2]:
!cp ../answer/{FILENAME} ./{FILENAME}

In [3]:
!clang++ {FILENAME} -std=gnu++17 -Wall -Wextra -O2 -o {FILENAME}.out -I/

In [2]:
SEED = 0
N_TESTCASES = 500

In [5]:
!seq {SEED} {SEED+N_TESTCASES-1} > seeds.txt && cargo run --release --manifest-path ../tools/Cargo.toml --bin gen seeds.txt
!ls in

    Finished release [optimized] target(s) in 0.19s
     Running `/home/user/ahc_httf_2022_qual/tools/target/release/gen seeds.txt`
0000.txt  0063.txt  0126.txt  0189.txt	0252.txt  0315.txt  0378.txt  0441.txt
0001.txt  0064.txt  0127.txt  0190.txt	0253.txt  0316.txt  0379.txt  0442.txt
0002.txt  0065.txt  0128.txt  0191.txt	0254.txt  0317.txt  0380.txt  0443.txt
0003.txt  0066.txt  0129.txt  0192.txt	0255.txt  0318.txt  0381.txt  0444.txt
0004.txt  0067.txt  0130.txt  0193.txt	0256.txt  0319.txt  0382.txt  0445.txt
0005.txt  0068.txt  0131.txt  0194.txt	0257.txt  0320.txt  0383.txt  0446.txt
0006.txt  0069.txt  0132.txt  0195.txt	0258.txt  0321.txt  0384.txt  0447.txt
0007.txt  0070.txt  0133.txt  0196.txt	0259.txt  0322.txt  0385.txt  0448.txt
0008.txt  0071.txt  0134.txt  0197.txt	0260.txt  0323.txt  0386.txt  0449.txt
0009.txt  0072.txt  0135.txt  0198.txt	0261.txt  0324.txt  0387.txt  0450.txt
0010.txt  0073.txt  0136.txt  0199.txt	0262.txt  0325.txt  0388.txt  0451.txt
0011.txt  

In [ ]:
# %%time
# # もとのスコアの確認
# scores = []
# for i in range(N_TESTCASES):
#     i = f"{i:04d}"
#     score = !../tools/target/release/tester $(pwd)/{FILENAME}.out < in/{i}.txt 2>&1 | grep Score
#     print(i, *score)
#     score = int(score[0].split()[-1])
#     scores.append(score)
# np.mean(scores), np.mean(scores) * 50, np.std(scores)

In [3]:
# ここでエラー出力は何かおかしいかもしれない


# パラメータ抽出
with open(FILENAME) as f:
    answer = f.read()


variables_optimize = []

for left, value, right, options in re.findall(r"^([^/\n]*=\s*)(.+?)(\s*;\s*//(?:.*\W)?OPTIMIZE(\W.*))$", answer, re.MULTILINE):
    name = left.replace("=", "").strip().split()[-1]
    
    searched = re.search(r".*\[(?P<min>.*),(?P<max>.*)\].*", options)
    if searched:
        min_value = max_value = None
        try:
            min_value = eval(searched.group("min"))
            max_value = eval(searched.group("max"))
            assert min_value <= max_value
        except Exception as e:
            print(f"searched={searched}", file=sys.stderr)
            print(e, file=sys.stderr)
            continue
        log = "LOG" in options  # 雑、直したほうが良い
        if type(min_value) != type(max_value):
            print(f"searched={searched}", file=sys.stderr)
            print("types not matched", file=sys.stderr)
            continue
        if isinstance(min_value, int):
            method = "suggest_int"
        elif isinstance(min_value, float):
            method = "suggest_float"
        else:
            print(f"searched={searched}", file=sys.stderr)
            print(f"unknown type ({type(min_value)})", file=sys.stderr)
            continue
        variables_optimize.append({
            "name": name,
            "method": method,
            "min": min_value,
            "max": max_value,
            "log": log,
            "left": left,
            "right": right,
        })
    elif searched := re.search(r".*\{(?P<choices>.*?)\}.*", options):
        choices = list(map(lambda x: x.strip(), searched.group("choices").split(",")))
        variables_optimize.append({
            "name": name,
            "method": "suggest_categorical",
            "choices": choices,
            "left": left,
            "right": right,
        })
    else:
        print(f"searched={searched}", file=sys.stderr)
        print(f"pattern was matched but options are incorrect.", file=sys.stderr)

print(len(variables_optimize), "variables were found.")
if globals().get("pd"):
    display(pd.DataFrame(variables_optimize))
else:
    pprint(variables_optimize)



6 variables were found.


,name,method,min,max,log,left,right
0,MCMC_Q_L2_NORM_RANGE,suggest_float,2.000,20.0,True,constexpr auto MCMC_Q_L2_NORM_RANGE =,"; // OPTIMIZE LOG [2.0, 20.0]"
1,MCMC_Q_RANGE,suggest_float,0.200,10.0,True,constexpr auto MCMC_Q_RANGE =,"; // OPTIMIZE LOG [0.2, 10.0]"
2,EXPECTED_SKILL_EMA_ALPHA_COEF,suggest_float,0.020,2.0,True,constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF =,"; // OPTIMIZE LOG [0.02, 2.0]"
3,MAX_N_NOT_OPEN_TASKS_IN_QUEUE,suggest_int,60.000,100.0,False,constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE =,"; // OPTIMIZE [60, 100]"
4,PRIORITY_DAY_OFFSET,suggest_int,400.000,1200.0,False,constexpr auto PRIORITY_DAY_OFFSET =,"; // OPTIMIZE [400, 1200]"
5,PRIORITY_COEF,suggest_float,0.002,2.0,True,constexpr auto PRIORITY_COEF =,"; // OPTIMIZE LOG [0.002, 2.0]"


In [4]:
def escape(string):  # 正規表現の中でそのまま使いたい文字列をエスケープ
    res = !echo '{string}' | sed -e 's/[]\/$*.^[]/\\&/g'
    return res[0]

def escape_sed(string):  # sed の置換後の文字列用のエスケープ
    res = !echo '{string}' | sed -e 's/[\/&]/\\&/g'
    return res[0]

def run(cmd):
    p = subprocess.Popen(
        cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=isinstance(cmd, str)
    )
    for line in iter(p.stdout.readline, b''):
        print(line.decode("utf8"), end="")

def objective_with_index(trial, n_internal_parallel):
    index_parallel = f"{trial.number:04d}"
    print(f"{index_parallel=}")
    
    parameters_changed_filename = f"{index_parallel}_{FILENAME}"
    directory_input = Path("./in")  # 中のすべてのファイルに対して実行される
    
    run(["mkdir", f"{index_parallel}_out"])
    run(["mkdir", f"{index_parallel}_score"])
    
    # ファイル作成
    run(f"cp {FILENAME} {parameters_changed_filename}")
    sed_options = [f"-i {parameters_changed_filename}"]
    for variable in variables_optimize:
        if variable["method"] == "suggest_categorical":
            val =  trial.suggest_categorical(variable["name"], variable["choices"])
        else:
            val = getattr(trial, variable["method"])(variable["name"], variable["min"], variable["max"], log=variable["log"])
        left = variable["left"]
        right = variable["right"]
        sed_options.append(f"""-e 's/^{escape(left)}.*{escape(right)}$/{escape_sed(left)}{val}{escape_sed(right)}/'""")
    command_sed = f"sed {' '.join(sed_options)}"
    print(command_sed)
    run(command_sed)
    
    # コンパイル
    command_compile = f"clang++ {parameters_changed_filename} -std=gnu++17 -O2 -DONLINE_JUDGE -I/ -o {parameters_changed_filename}.out"
    print(command_compile)
    run(command_compile)
    # 実行・採点コマンド (@ はファイル名)
    command_exec = (
        #f"../tools/target/release/tester $(pwd)/{parameters_changed_filename}.out < {directory_input}/@ 2>&1 | grep Score"
        f"../tools/target/release/tester $(pwd)/{parameters_changed_filename}.out < {directory_input}/@ 2>&1 | grep Score | sed -E s/[^0-9]+// > ./{index_parallel}_score/@;"
        #f"cargo run --release --manifest-path ../tools/Cargo.toml --bin tester {directory_input}/@ $(pwd)/{parameters_changed_filename}.out 2>&1 | grep Score | sed -E s/[^0-9]+// > ./{index_parallel}_score/@;"
        #f"./{parameters_changed_filename}.out < {directory_input}/@ > ./{index_parallel}_out/@;"
        #f"cargo run --release --manifest-path ./tools/Cargo.toml --bin vis {directory_input}/@ ./{index_parallel}_out/@ 2> /dev/null > ./{index_parallel}_score/@;"
    )
    # 並列実行 (sed はパスのディレクトリ部分を消してファイル名にしてる)
    run(f"find {directory_input}/* | sed 's!^.*/!!' | xargs -I@ -P {n_internal_parallel} sh -c '{command_exec}'")
    
    # 集計
    sum_score = 0
    for file_path in Path(f"./{index_parallel}_score/").iterdir():  
        with open(file_path) as f:
            sum_score += int(f.read())
    
    # 後始末
    run(f"rm -rf {index_parallel}_out")
    run(f"rm -rf {index_parallel}_score")
#     run(f"rm {parameters_changed_filename}")
    run(f"rm {parameters_changed_filename}.out")
    
    return sum_score


In [5]:
storage_path = f"study.db"
storage = f"sqlite:///{storage_path}"
study_name = "study"
study = optuna.create_study(storage=storage, load_if_exists=True, study_name=study_name, direction="maximize")

def callback(study, trial):
    try:
        index_parallel = f"{trial.number:04d}"
        parameters_changed_filename = f"{index_parallel}_{FILENAME}"
        if study.best_value == trial.value:
            print(f"Updated! {study.best_value}")
            !cp {parameters_changed_filename} 00000_best_parameters.cpp
        !rm {parameters_changed_filename}
        print("callback ok")
    except:
        print(":(")

[I 2021-11-13 03:44:07,063] Using an existing study with name 'study' instead of creating a new one.


In [6]:
objective = partial(objective_with_index, n_internal_parallel=8)
study.optimize(objective, n_trials=3000, timeout=86400, n_jobs=3, callbacks=[callback])

/home/user/anaconda3/lib/python3.8/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(


index_parallel='0132'
index_parallel='0133'
index_parallel='0134'
sed -i 0132_main.cpp -e 's/^constexpr auto MCMC_Q_L2_NORM_RANGE = .*;           \/\/ OPTIMIZE LOG \[2\.0, 20\.0\]$/constexpr auto MCMC_Q_L2_NORM_RANGE = 8.62848995348339;           \/\/ OPTIMIZE LOG [2.0, 20.0]/' -e 's/^constexpr auto MCMC_Q_RANGE = .*;                   \/\/ OPTIMIZE LOG \[0\.2, 10\.0\]$/constexpr auto MCMC_Q_RANGE = 1.145286155886507;                   \/\/ OPTIMIZE LOG [0.2, 10.0]/' -e 's/^constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = .*; \/\/ OPTIMIZE LOG \[0\.02, 2\.0\]$/constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = 0.7223285839117831; \/\/ OPTIMIZE LOG [0.02, 2.0]/' -e 's/^constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = .*; \/\/ OPTIMIZE \[60, 100\]$/constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = 94; \/\/ OPTIMIZE [60, 100]/' -e 's/^constexpr auto PRIORITY_DAY_OFFSET = .*;          \/\/ OPTIMIZE \[400, 1200\]$/constexpr auto PRIORITY_DAY_OFFSET = 714;          \/\/ OPTIMIZE [400, 1200]/' -e 's/^co

[I 2021-11-13 03:48:29,156] Trial 134 finished with value: 1011549.0 and parameters: {'MCMC_Q_L2_NORM_RANGE': 12.540600636679095, 'MCMC_Q_RANGE': 1.1236784084262321, 'EXPECTED_SKILL_EMA_ALPHA_COEF': 0.3852669161058527, 'MAX_N_NOT_OPEN_TASKS_IN_QUEUE': 94, 'PRIORITY_DAY_OFFSET': 723, 'PRIORITY_COEF': 0.21339683565413894}. Best is trial 47 with value: 1013922.0.


callback ok
index_parallel='0135'


[I 2021-11-13 03:48:29,677] Trial 133 finished with value: 1012616.0 and parameters: {'MCMC_Q_L2_NORM_RANGE': 12.865789706938177, 'MCMC_Q_RANGE': 1.1503023664694032, 'EXPECTED_SKILL_EMA_ALPHA_COEF': 0.6803347047866688, 'MAX_N_NOT_OPEN_TASKS_IN_QUEUE': 94, 'PRIORITY_DAY_OFFSET': 585, 'PRIORITY_COEF': 0.22317332801370435}. Best is trial 47 with value: 1013922.0.


...

callback ok
index_parallel='0265'
sed -i 0265_main.cpp -e 's/^constexpr auto MCMC_Q_L2_NORM_RANGE = .*;           \/\/ OPTIMIZE LOG \[2\.0, 20\.0\]$/constexpr auto MCMC_Q_L2_NORM_RANGE = 8.118252477470673;           \/\/ OPTIMIZE LOG [2.0, 20.0]/' -e 's/^constexpr auto MCMC_Q_RANGE = .*;                   \/\/ OPTIMIZE LOG \[0\.2, 10\.0\]$/constexpr auto MCMC_Q_RANGE = 1.0592428685894035;                   \/\/ OPTIMIZE LOG [0.2, 10.0]/' -e 's/^constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = .*; \/\/ OPTIMIZE LOG \[0\.02, 2\.0\]$/constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = 0.49150488552347094; \/\/ OPTIMIZE LOG [0.02, 2.0]/' -e 's/^constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = .*; \/\/ OPTIMIZE \[60, 100\]$/constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = 97; \/\/ OPTIMIZE [60, 100]/' -e 's/^constexpr auto PRIORITY_DAY_OFFSET = .*;          \/\/ OPTIMIZE \[400, 1200\]$/constexpr auto PRIORITY_DAY_OFFSET = 651;          \/\/ OPTIMIZE [400, 1200]/' -e 's/^constexpr auto PRIORITY_COEF = 

[I 2021-11-13 06:51:53,962] Trial 263 finished with value: 1011345.0 and parameters: {'MCMC_Q_L2_NORM_RANGE': 7.4698687662137395, 'MCMC_Q_RANGE': 1.0344748808008521, 'EXPECTED_SKILL_EMA_ALPHA_COEF': 0.590189600971181, 'MAX_N_NOT_OPEN_TASKS_IN_QUEUE': 98, 'PRIORITY_DAY_OFFSET': 646, 'PRIORITY_COEF': 0.3801831735062558}. Best is trial 211 with value: 1013959.0.


callback ok
index_parallel='0266'
sed -i 0266_main.cpp -e 's/^constexpr auto MCMC_Q_L2_NORM_RANGE = .*;           \/\/ OPTIMIZE LOG \[2\.0, 20\.0\]$/constexpr auto MCMC_Q_L2_NORM_RANGE = 8.10800372584208;           \/\/ OPTIMIZE LOG [2.0, 20.0]/' -e 's/^constexpr auto MCMC_Q_RANGE = .*;                   \/\/ OPTIMIZE LOG \[0\.2, 10\.0\]$/constexpr auto MCMC_Q_RANGE = 1.2354936295558527;                   \/\/ OPTIMIZE LOG [0.2, 10.0]/' -e 's/^constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = .*; \/\/ OPTIMIZE LOG \[0\.02, 2\.0\]$/constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = 0.47563854485759766; \/\/ OPTIMIZE LOG [0.02, 2.0]/' -e 's/^constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = .*; \/\/ OPTIMIZE \[60, 100\]$/constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = 97; \/\/ OPTIMIZE [60, 100]/' -e 's/^constexpr auto PRIORITY_DAY_OFFSET = .*;          \/\/ OPTIMIZE \[400, 1200\]$/constexpr auto PRIORITY_DAY_OFFSET = 647;          \/\/ OPTIMIZE [400, 1200]/' -e 's/^constexpr auto PRIORITY_COEF = .

[W 2021-11-13 06:54:40,303] Trial 265 failed because of the following error: ValueError("invalid literal for int() with base 10: ''")
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.8/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-4-72e45d5d6fbc>", line 60, in objective_with_index
    sum_score += int(f.read())
ValueError: invalid literal for int() with base 10: ''
[W 2021-11-13 06:54:40,317] Trial 266 failed because of the following error: ValueError("invalid literal for int() with base 10: ''")
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.8/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-4-72e45d5d6fbc>", line 60, in objective_with_index
    sum_score += int(f.read())
ValueError: invalid literal for int() with base 10: ''
[W 2021-11-13 06:54:40,318] Trial 264 failed because of 

KeyboardInterrupt: 